GLM Z-Score Analysis
=====================================

This script gets z-scores and p-values for the Schaefer atlas nodes by

1.  loading the second-level GLM z-maps for each contrast, 
2.  calculating the average value for each node in the Schaefer atlas, 
3.  computing the z-score and p-values for each node in the Schaefer atlas, and saving the z-scores and p-values into a new npz file.

# Get hub activation for social and non-social tasks

1. Load subject info (cond order list) and atlas info
2. Get dict of hubs with ROI indices
3. Create mask for each set of hubs that you want to extract average activation from
4. Load transition versus non-transition nii file for each task
5. Extract average activation in mask for each set of hubs
6. Loop over each subject & task and fill empty pandas dataframe
7. Save pandas df as CSV

In [ ]:
#importing libraries
%matplotlib inline
from __future__ import print_function, division
import scipy.stats as ss
import os
from os.path import join as opj
import pandas as pd
import numpy as np
from numpy import inf
import matplotlib.pyplot as plt
import seaborn as sns
from nilearn import image, plotting, input_data
import nibabel as nb
from mne.stats import fdr_correction
from nistats import thresholding
from random import shuffle
import glob

print('Done Importing Packages')

In [ ]:
# Change to directory where you saved the data
home_path = '/Users/steventompson/Git/tompson_netlearn_fmri'

data_dir = opj(home_path,'data')
template_dir = opj(data_dir,'brain_atlas')
path_InpData = opj(data_dir,'Subject_Data','netLearn_glm','firstLevel')
path_zData = opj(data_dir,'ppi_zscores')
path_OutpData = opj(data_dir,'glm_means')
path_Figures = opj(home_path,'figures','component_figs') # folder to put figures


for path in [path_OutpData, path_Figures]:
    if not os.path.exists(path):
        print('Path does not exist, creating {}'.format(path))
        os.makedirs(path)

print('Set data paths')

In [ ]:
subj_links=pd.read_csv('{}/subj_data/netLearn_IDs_26subjs.csv'.format(data_dir))

bad_subjs = ['SNL_001','SNL_004','SNL_028']
subjs = [s for s in subj_links.loc[:,'scanID'].tolist() if s not in bad_subjs]
#subjs.reverse()
n_subjs = len(subjs)
print('We have %d subjects' % (n_subjs))

In [ ]:
#load schaefer mask
sch_filename='{}/schaefer400_harvard_oxford_2mm_mni_17network.nii.gz'.format(template_dir)
schaefer_mask=nb.load(sch_filename)

#load schaefer atlas info
schaefer_atlas=pd.read_csv('{}/s400ho_netLearn_2mm.csv'.format(template_dir))

schaefer_atlas=schaefer_atlas.fillna('Uncertain')
schaefer_atlas.loc[schaefer_atlas['System']=='Uncertain','System']='Subcortical'

schaefer_atlas.loc[[402,407],'System']='Hippocampus'
schaefer_atlas.loc[[402,407],'System7']='Hippocampus'

sch_names=np.unique(schaefer_atlas['System'])
sch_nums=[int(np.where(sch_names==label)[0]) for label in schaefer_atlas['System']]


schaefer_atlas.loc[schaefer_atlas['System7']=='Uncertain','System7']='Subcortical'
sch7_names=np.unique(schaefer_atlas['System7'])
sch7_nums=[int(np.where(sch7_names==label)[0]) for label in schaefer_atlas['System7']]

net_coords=np.array(schaefer_atlas.loc[:,['x','y','z']])
net_cols=['black']*len(net_coords)

n_node = len(sch_nums)
triu_ix, triu_iy = np.triu_indices(n_node, k=1)
n_conn = len(triu_ix)

n_perm = 500

schaefer_atlas.head()

In [ ]:
subj_links.loc[:,'nsFile']='NA'
subj_links.loc[:,'socFile']='NA'
for subj in subj_links['pID']:
    cond=subj_links.loc[subj_links['pID']==subj,'CondNum'].tolist()[0]
    scanID=subj_links.loc[subj_links['pID']==subj,'scanID'].tolist()[0]
    if cond==1:
        nsFile='{}_task1_Transition_z_map.nii.gz'.format(scanID)
        socFile='{}_task2_Transition_z_map.nii.gz'.format(scanID)
    else:
        nsFile='{}_task2_Transition_z_map.nii.gz'.format(scanID)
        socFile='{}_task1_Transition_z_map.nii.gz'.format(scanID)
    subj_links.loc[subj_links['pID']==subj,'nsFile']=nsFile
    subj_links.loc[subj_links['pID']==subj,'socFile']=socFile
    
ns_filenames=subj_links['nsFile'].values
soc_filenames=subj_links['socFile'].values

In [ ]:

list1=['combData','diffData','nonSocData','SocData']
list1_labels=['All Tasks','Social versus Non-Social','Non-Social Task','Social Task']
list2=['nodestr']
list2_labels=['Global']


apriori_names=['dmPFC_L','vmPFC_L','PCC_L','Hippocampus_L','TPJ_L',
               'dmPFC_R','vmPFC_R','PCC_R','Hippocampus_R','TPJ_R',
               'Frontal_Inf_Oper_L','Frontal_Inf_Orb_L','Frontal_Inf_Tri_L',
               'Frontal_Inf_Oper_R','Frontal_Inf_Orb_R','Frontal_Inf_Tri_R',
               'Amygdala_L','Amygdala_R',
               'Ventral_Striatum_L','Caudate_L',
               'Ventral_Striatum_R','Caudate_R']



def identify_hubs1(ix,iy,thresh=True,apriori=True,flip=False,alpha=0.05):
    xx=list2[ix]
    yy=list1[iy]
    data=np.load('{}/netLearn_{}_zscores_26subjs_{}.npz'.format(path_zData,yy,xx))
    zMat=data['zMat']
    pMat=data['pMat']
    #pMat=np.multiply(data['pMat'],2)
    
    if flip:
        zMat=np.multiply(zMat,-1)
    
    # Set nonsignificant values to zero using FDR correction
    if thresh:
        reject_fdr,pval_fdr=fdr_correction(pMat,alpha)
        zMat=np.multiply(zMat,reject_fdr)

    title='{}- {} Connectivity'.format(list1_labels[iy],list2_labels[ix])
    sig_ix=np.where(zMat>0)[0]
    sig_dic={}
    sig_vec=np.zeros(zMat.shape)
    if apriori:
        sig_ix=[x for x in sig_ix if schaefer_atlas.loc[x,'ns_ROI_label'] in apriori_names]
        for i,x in enumerate(apriori_names):
            sig_dic[x]=[ix for ix in sig_ix if schaefer_atlas.loc[ix,'ns_ROI_label']==x]
            sig_dic=dict((k, v) for k, v in sig_dic.iteritems() if v)

    sig_vec[sig_ix]=1
        
    print('')
    print(title)
    print('{} significant positive ROIs in a priori regions'.format(len(sig_ix)))
    return sig_ix,sig_vec,sig_dic
    
sigvals_comb,sigvec_comb,sigdic_comb=identify_hubs1(0,0,alpha=0.025)
signames_comb=schaefer_atlas.loc[sigvals_comb,'ns_ROI_label']

sigvals_diff,sigvec_diff,sigdic_diff=identify_hubs1(0,1,alpha=0.025)
signames_diff=schaefer_atlas.loc[sigvals_diff,'ns_ROI_label']


sigvals_diff_ns,sigvec_diff_ns,sigdic_diff_ns=identify_hubs1(0,1,flip=True,alpha=0.025)
signames_diff_ns=schaefer_atlas.loc[sigvals_diff_ns,'ns_ROI_label']

sigdic_comb['Hippocampus']=[402,407]
sigdic_comb['combhubs']=sigvals_comb
sigdic_diff['diffhubs']=sigvals_diff
sigdic_diff_ns['diffhubs_ns']=sigvals_diff_ns

lpfc_l_vals=[sigdic_diff_ns[key] for key in sigdic_diff_ns.keys() if 'Frontal' in key and '_L' in key]
sigdic_diff_ns['lPFC_L']=[item for sublist in lpfc_l_vals for item in sublist]
lpfc_r_vals=[sigdic_diff_ns[key] for key in sigdic_diff_ns.keys() if 'Frontal' in key and '_R' in key]
sigdic_diff_ns['lPFC_R']=[item for sublist in lpfc_r_vals for item in sublist]


for x in sigdic_comb.keys():
    print('DG Hubs: Column ids for {}= {}'.format(x,sigdic_comb[x]))
    
print('')
for x in sigdic_diff.keys():
    print('Soc Hubs: Column ids for {}= {}'.format(x,sigdic_diff[x]))
    
print('')
for x in sigdic_diff_ns.keys():
    print('NS Hubs: Column ids for {}= {}'.format(x,sigdic_diff_ns[x]))

In [ ]:
def get_atlas_means(flist,fpath,atlas_path):
    fnames=[opj(fpath,f) for f in flist]
    print('Concatenating {} images'.format(len(fnames)))
    all_imgs=image.concat_imgs(fnames,memory='nilearn_cache',memory_level=1)
    masker = input_data.NiftiLabelsMasker(atlas_path,
                                           detrend=False, 
                                           standardize=False, 
                                           low_pass=None, high_pass=None, 
                                           t_r=1,
                                           memory='nilearn_cache', memory_level=1); #verbose=2 by default nothing should be printed

    print('Extracting values from nii image')
    all_ts = masker.fit_transform(all_imgs).squeeze()

    print('Matrix created with shape {}'.format(all_ts.shape))
    return all_ts

# In[ ]:
#adj_mat is 2-d matrix with n_subj x n_node shape
#row_ix is a list of indices to keep for the rows
#col_ix is a list of indices to keep for the columns
def get_means(roi_mat,row_ix,col_ix):
    #cfg_mat is a 3D n_subjs x n_perm x n_conn np array
    #row_ix is a list of the row indices to include in the means to extract
    #col_ix is a list of the column indices to include in the means to extract
    subdata=roi_mat[row_ix,:]
    subdata=subdata[:,col_ix]
    rowMeans=np.mean(subdata,axis=(-1))
    return rowMeans

def get_subhub_means(roi_mat,col_dict,col_id):
    #roi_cfg is a 2D n_subjs x n_conn np array
    #nulll_cfg is a 3D n_subjs x n_perm x n_conn np array
    #row_id is a string that matches a key in the row_dict (or is 'all' which will get global connectivity)
    #row_dict is a dictionary with keys matching a priori hubs and row indices for each hub
    #col_id is a string that matches a key in the col_dict (or is 'all' which will get global connectivity)
    #col_dict is a dictionary with keys matching a priori hubs and column indices for each hub
    #thresh is a boolean determining whether to set non-significant z-scores to zero 
    if col_id=='all':
        col_ix=range(410)
    else:
        col_ix=col_dict[col_id]
        
    print('Getting subject averages for {}'.format(col_id))
    subj_vec=get_means(roi_mat,range(roi_mat.shape[0]),col_ix)
    return subj_vec
    
def subj_activation(file_list,file_path,atlas_file,savename,savepath):

    # load activation maps and get mean activation for each node in atlas
    node_means=get_atlas_means(file_list,file_path,atlas_file)
    
    #df=pd.DataFrame({'pID':[f[:7] for f in file_list]})
    df=pd.DataFrame()
    
    for a in ['combhubs','Hippocampus_L','Hippocampus_R']:
    #for a in ['Hippocampus','Hippocampus_L','Hippocampus_R','Frontal_Inf_Orb_L','Frontal_Inf_Tri_R','dmPFC_L','dmPFC_R','vmPFC_L']:
            colname='{}_glm'.format(a)
            sub_means=get_subhub_means(node_means,sigdic_comb,a)
            df[colname]=sub_means

    for a in ['diffhubs','TPJ_L','TPJ_R']:
            colname='{}_glm'.format(a)
            sub_means=get_subhub_means(node_means,sigdic_diff,a)
            df[colname]=sub_means
            
    for a in ['diffhubs_ns','dmPFC_L','dmPFC_R','lPFC_L','lPFC_R']:
            colname='{}_glm'.format(a)
            sub_means=get_subhub_means(node_means,sigdic_diff_ns,a)
            df[colname]=sub_means
     
    save_file='{}/netLearn_{}_26subjs_sighubs_activation.csv'.format(savepath,savename)
    df.to_csv(save_file)
    print('saving df with shape {} to file: {}'.format(df.shape,opj(save_file)))
    return df

In [ ]:
ns_df=subj_activation(file_list=ns_filenames,file_path=path_InpData,atlas_file=sch_filename,savename='nsData_glm_transition',savepath=path_OutpData)
soc_df=subj_activation(file_list=soc_filenames,file_path=path_InpData,atlas_file=sch_filename,savename='socData_glm_transition',savepath=path_OutpData)

In [ ]:
ns_df.mean()

In [ ]:
soc_df.mean()

Steven Tompson | 2019